### scvi must be version 0.6.7

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
from scvi.models import VAE, SCANVI
from scvi.inference import UnsupervisedTrainer, SemiSupervisedTrainer
from sklearn.preprocessing import LabelEncoder
from scvi.dataset import AnnDatasetFromAnnData
import numpy as np


import scvi
import scanpy as sc
sc.set_figure_params(figsize=(4, 4))

import sys

#if True, will install via pypi, else will install from source
stable = True
IN_COLAB = "google.colab" in sys.modules

if IN_COLAB and stable:
    !pip install --quiet scvi-tools[tutorials]
elif IN_COLAB and not stable:
    !pip install --quiet --upgrade jsonschema
    !pip install --quiet git+https://github.com/yoseflab/scvi-tools@master#egg=scvi-tools[tutorials]

# to autoreload upon editing functions
%load_ext autoreload

from matplotlib import rcParams
import matplotlib.pyplot as plt
%matplotlib inline

from scvi.models import VAE
from scvi.inference import UnsupervisedTrainer
from sklearn.preprocessing import LabelEncoder
from scvi.dataset import AnnDatasetFromAnnData


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
print('the latest used version before revision was 0.6.7')

the latest used version before revision was 0.6.7


In [4]:
scvi.__version__

'0.6.7'

In [5]:
cd ..

/storage/groups/ml01/workspace/ignacio.ibarra/theislab/saliba_macrophages/notebooks


In [6]:

from utils import *

In [7]:
adata = get_concatenated_dataset()

adams
scoring MP.markers 20
scoring MP.others 9
scoring MP.all 30
morse


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Vari

scoring MP.markers 20
scoring MP.others 10
scoring MP.all 31
bal


Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Variable names are not unique. To make them unique, call `.var_names_make_unique`.
Trying to set attribute `.obs` of view, copying.


adding annotation from bal mdm...
scoring genes...
scoring MP.markers 20
scoring MP.others 10
scoring MP.all 31
budinger
scoring MP.markers 20
scoring MP.others 10
scoring MP.all 31
14923.0
10239.0
22851.0
6783.0
morse+adams
(morse+adams) + bal
(morse+adams+bal) + budinger
False    11774
True      5882
Name: highly_variable, dtype: int64


In [8]:
cd revision

/storage/groups/ml01/workspace/ignacio.ibarra/theislab/saliba_macrophages/notebooks/revision


In [9]:
print('datasets loaded...')

datasets loaded...


In [10]:
# Check for counts data layer
if 'counts' not in adata.layers:
    raise TypeError('Adata does not contain a `counts` layer in `adata.layers[`counts`]`')

In [11]:
adata

AnnData object with n_obs × n_vars = 227086 × 17656
    obs: 'GSM', 'sample.id', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'patient.id', 'cell.type', 'study', 'disease.status', 'MP.markers.score', 'MP.others.score', 'MP.all.score', 'nUMI', 'nGene', 'CellType_Category', 'Manuscript_Identity', 'Subclass_Cell_Identity', 'Disease_Identity', 'Subject_Identity', 'Library_Identity', 'groups', 'batch', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'viral_counts', 'viral_ratio', 'viral_norm', 'percent.mt', 'number', 'dataset', 'S.Score', 'G2M.Score', 'Phase', 'CC.Difference', 'SR.score1', 'dissoc.score1', 'Label', 'nCount_VIRAL', 'nFeature_VIRAL', 'seurat_clusters', 'RNA_mnn_40_Leiden_res.0.1', 'RNA_mnn_40_Leiden_res.0.2', 'RNA_mnn_40_Leiden_res.0.3', 'RNA_mnn_40_Leiden_res.0.4', 'RNA_mnn_40_Leiden_res.0.5', 'RNA_mnn_40_Leiden_res.0.6', 'RNA_mnn_40_Leiden_res.0.7', 'RNA_mnn_40_Leiden_res.0.8', 'RNA_mnn_40_Leiden_res.0.9', 'RNA_mnn_40_Leiden_res.1', 'RNA_mnn_40_Leiden_res.1.5', 'RNA_mnn_40_L

In [12]:
adata.obs['study'].value_counts()

adams       179222
budinger     22810
morse        17551
BAL           7503
Name: study, dtype: int64

In [13]:
adata.obs[adata.obs['study'] == 'morse']['patient.id'].value_counts()

SC156    2104
SC45     1935
SC59     1743
SC155    1733
SC88     1519
SC87     1371
SC154    1323
SC89     1223
SC153    1099
SC56     1080
SC94      842
SC93      509
SC95      415
SC31D     239
SC31      213
SC14      203
Name: patient.id, dtype: int64

### The batch correction will be attempted by patient

In [14]:
adata.obs['study'].value_counts()

adams       179222
budinger     22810
morse        17551
BAL           7503
Name: study, dtype: int64

In [15]:
batch = 'patient.id'
labels = 'study'

In [16]:
adata.obs['study'].value_counts()

adams       179222
budinger     22810
morse        17551
BAL           7503
Name: study, dtype: int64

### Update 03.17.2021
To verify whether the association between mdm and IPF is due to an oversampling of mdm from Adams, we are now sampling an equal proportion of IPF and control cells from that study.

In [18]:
adata.obs

,GSM,sample.id,n_counts,log_counts,n_genes,mt_frac,patient.id,cell.type,study,disease.status,...,Diagnosis,COVID-19,Tissue Type,Cell Population,Cell Type,leiden,No of UMIs,No of genes,% of mito genes,Cluster
SC154IPFUP_AAACCTGCATCACGAT-1-0-0-0,GSM3660658,SC154IPFUP,15616.0,9.656052,2600.0,0.093558,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGGTACCGGCT-1-0-0-0,GSM3660658,SC154IPFUP,9768.0,9.186867,2008.0,0.084562,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGTCTGTCCGT-1-0-0-0,GSM3660658,SC154IPFUP,27659.0,10.227707,3509.0,0.043386,SC154,FABP4hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATCCCGC-1-0-0-0,GSM3660658,SC154IPFUP,9778.0,9.187890,2276.0,0.070157,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATGCGAC-1-0-0-0,GSM3660658,SC154IPFUP,8610.0,9.060679,1981.0,0.059582,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
152653-1,NaN,NaN,11819.0,9.377463,2883.0,0.000000,PMB 2,M3,budinger,covid,...,COVID-19,True,Parenchyma,Immune-TX,MoAM-2,2.0,11819.0,2883.0,4.095101,MoM3
152916-1,NaN,NaN,3450.0,8.146130,1292.0,0.000000,PMB 2,M3,budinger,covid,...,COVID-19,True,Parenchyma,Immune-TX,MoAM-3,4.0,3450.0,1292.0,6.927536,MoM3
153517-1,NaN,NaN,10060.0,9.216323,2717.0,0.001292,PMB 2,M3,budinger,covid,...,COVID-19,True,Parenchyma,Immune-TX,MoAM-3,4.0,10060.0,2717.0,5.248509,MoM3
154761-1,NaN,NaN,10598.0,9.268420,2505.0,0.000000,PMB 2,M3,budinger,covid,...,COVID-19,True,Parenchyma,Immune-TX,MoAM-3,4.0,10598.0,2505.0,10.983204,MoM3


In [19]:
# print('subsetting...')
# ad = subset_anndata(adata, N=adata.shape[0])
# print('done.')

In [30]:
adata.obs.groupby(['study', 'disease.status']).size()


study     disease.status
BAL       BAL (NA)           7503
adams     COPD              30159
          IPF               86002
          control           63061
budinger  covid             22810
morse     IPF                8301
          control            9250
dtype: int64

In [53]:
import random

adams_copd = adata[(adata.obs['study'] == 'adams') & (adata.obs['disease.status'] == 'COPD'),:]
adams_ipf = adata[(adata.obs['study'] == 'adams') & (adata.obs['disease.status'] == 'IPF'),:]
adams_control = adata[(adata.obs['study'] == 'adams') & (adata.obs['disease.status'] == 'control'),:]


# sample N_COPD cells from IPF and from control
n_copd = adams_copd.shape[0]
random.seed(500)
adams_ipf = adams_ipf[adams_ipf.obs.index.isin(set(random.sample(list(adams_ipf.obs.index), n_copd))),:]
adams_control = adams_control[adams_control.obs.index.isin(set(random.sample(list(adams_control.obs.index), n_copd))),:]


adams_copd.shape, adams_ipf.shape, adams_control.shape

adata = adata[adata.obs.study != 'adams',].concatenate(adams_copd).concatenate(adams_ipf).concatenate(adams_control)

adata.obs.groupby(['study', 'disease.status']).size()

In [58]:
ad = adata

In [59]:
ad.shape

(138341, 17656)

In [60]:
marker_genes = get_marker_genes_ipf()
ad.var['hvg.or.marker'] = ad.var['highly_variable'] | ad.var.index.isin(marker_genes['MP.all'])

In [61]:
ad.var['hvg.or.marker'].value_counts()

False    11769
True      5887
Name: hvg.or.marker, dtype: int64

### Data preparation and iterative training.

In [62]:
batch = 'patient.id'
# batch = 'study'

In [63]:
ad.obs[batch].value_counts()

Case 1    12258
178CO      6283
PMB 1      5309
PMB 2      5243
226C       4227
          ...  
084C        157
8CO         147
59I         115
244C         17
137CO        12
Name: patient.id, Length: 104, dtype: int64

In [64]:
adata.n_obs

138341

In [65]:
adata.obs

,GSM,sample.id,n_counts,log_counts,n_genes,mt_frac,patient.id,cell.type,study,disease.status,...,Diagnosis,COVID-19,Tissue Type,Cell Population,Cell Type,leiden,No of UMIs,No of genes,% of mito genes,Cluster
SC154IPFUP_AAACCTGCATCACGAT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,15616.0,9.656052,2600.0,0.093558,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGGTACCGGCT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,9768.0,9.186867,2008.0,0.084562,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGTCTGTCCGT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,27659.0,10.227707,3509.0,0.043386,SC154,FABP4hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATCCCGC-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,9778.0,9.187890,2276.0,0.070157,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATGCGAC-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,8610.0,9.060679,1981.0,0.059582,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484C_TTGACTTTCATCTGCC-1-0-0-1,NaN,NaN,16249.0,9.695786,3934.0,0.044187,484C,Macrophage_Outlier_D,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTACTGCAAATTGCC-1-0-0-1,NaN,NaN,1851.0,7.523481,1096.0,0.044300,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTATGCCAATAGAGT-1-0-0-1,NaN,NaN,11975.0,9.390576,3415.0,0.046180,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTATGCTCGATAGAA-1-0-0-1,NaN,NaN,6138.0,8.722254,2506.0,0.061584,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
ad.shape

(138341, 17656)

In [67]:
ad.shape

(138341, 17656)

In [68]:
# Use non-normalized (count) data for scvi!
# Expects data only on HVGs
# Defaults from SCVI github tutorials scanpy_pbmc3k and harmonization
n_epochs=np.min([round((20000/adata.n_obs)*400), 400])


n_epochs = 400
print('# epochs %i' % n_epochs)



# epochs 400


In [69]:
print('here...')

here...


In [71]:
net_adata = ad[:,ad.var['hvg.or.marker']].copy()
print(net_adata.shape)
net_adata.X = net_adata.layers['counts'] # [:,ad.var['hvg.or.marker']]
del net_adata.layers['counts']
# Ensure that the raw counts are not accidentally used
del net_adata.raw # Note that this only works from anndata 0.7

(138341, 5887)


In [ ]:
n_latent=30
n_hidden=128
n_layers=2



# Define batch indices
le = LabelEncoder()
net_adata.obs['batch_indices'] = le.fit_transform(net_adata.obs[batch].values)

net_adata = AnnDatasetFromAnnData(net_adata)

vae = VAE(
    net_adata.nb_genes,
    reconstruction_loss='nb',
    n_batch=net_adata.n_batches,
    n_layers=n_layers,
    n_latent=n_latent,
    n_hidden=n_hidden,
)

trainer = UnsupervisedTrainer(
    vae,
    net_adata,
    train_size=1.0,
    use_cuda=True,
)

trainer.train(n_epochs=n_epochs, lr=1e-3)

full = trainer.create_posterior(trainer.model, net_adata, indices=np.arange(len(net_adata)))
latent, _, _ = full.sequential().get_latent()

ad.obsm['X_emb'] = latent



[2021-03-17 23:59:06,056] INFO - scvi.dataset.dataset | Remapping batch_indices to [0,N]
[2021-03-17 23:59:06,065] INFO - scvi.dataset.dataset | Remapping labels to [0,N]
[2021-03-17 23:59:07,917] INFO - scvi.dataset.dataset | Computing the library size for the new data
[2021-03-17 23:59:08,158] INFO - scvi.dataset.dataset | Downsampled from 138341 to 138341 cells
[2021-03-17 23:59:14,106] INFO - scvi.inference.inference | KL warmup for 400 epochs
training:  77%|███████▋  | 307/400 [1:52:14<32:58, 21.28s/it]  

In [ ]:
latent

In [ ]:
import torch

In [ ]:
torch.save(trainer.model.state_dict(), 'mytraining_all_revision.pt')



In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from matplotlib import rcParams
import matplotlib.pyplot as plt
rcParams['figure.figsize'] = [6, 6]
rcParams['figure.dpi'] = 110

from os.path import join, exists

k = 'adams'
output_path = join('data/figures/%s_revision.png' % k)
if exists(output_path):
    print('skip %s' % k)

if 'X_emb' in ad.obsm:
    print('neighbors... using %s' % 'X_emb' )
    sc.pp.neighbors(ad, use_rep='X_emb')
    sc.tl.umap(ad)
    print('umap...')



In [ ]:
import anndata
anndata.__version__

In [ ]:
import scanpy as sc
sc.__version__

In [106]:
import os; os.path.exists(outfilename)

False

In [107]:
ad.write(outfilename, compression='lzf')

... storing 'GSM' as categorical
... storing 'sample.id' as categorical
... storing 'patient.id' as categorical
... storing 'cell.type' as categorical
... storing 'study' as categorical
... storing 'disease.status' as categorical
... storing 'CellType_Category' as categorical
... storing 'Manuscript_Identity' as categorical
... storing 'Subclass_Cell_Identity' as categorical
... storing 'Disease_Identity' as categorical
... storing 'Subject_Identity' as categorical
... storing 'Library_Identity' as categorical
... storing 'groups' as categorical
... storing 'orig.ident' as categorical
... storing 'number' as categorical
... storing 'dataset' as categorical
... storing 'Phase' as categorical
... storing 'Label' as categorical
... storing 'seurat_clusters' as categorical
... storing 'RNA_mnn_40_Leiden_res.0.1' as categorical
... storing 'RNA_mnn_40_Leiden_res.0.2' as categorical
... storing 'RNA_mnn_40_Leiden_res.0.3' as categorical
... storing 'RNA_mnn_40_Leiden_res.0.4' as categorical


In [108]:
print('done...')

done...


In [100]:
'morse_n_adams_n_bal_scvi_%s_%i_%i_revision.h5da' % (batch, ad.shape[0], n_epochs)

'morse_n_adams_n_bal_scvi_patient.id_138341_400_revision.h5da'

In [101]:
outfilename = 'morse_n_adams_n_bal_scvi_%s_%i_%i_revision.h5da' % (batch, ad.shape[0], n_epochs)

In [ ]:
print('done...')

In [1]:
import scanpy as sc

In [2]:
print('here...')

here...


In [58]:
print(outfilename)

morse_n_adams_n_bal_scvi_patient.id_227086_400_revision.h5da


In [59]:
print('loading object...')
ad = sc.read_h5ad(outfilename)
print('done...')

loading object...
done...


In [82]:
ad.shape

(138341, 17656)

In [94]:
ad.obs[ad.obs['study'] == 'adams'][['patient.id', 'disease.status']]

,patient.id,disease.status
052CO-a_AAACCTGCAGGCTGAA-1-0-0-1-0-0,052CO,COPD
052CO-a_AAACCTGGTATCAGTC-1-0-0-1-0-0,052CO,COPD
052CO-a_AAACGGGAGTGCGATG-1-0-0-1-0-0,052CO,COPD
052CO-a_AAAGTAGGTCCATGAT-1-0-0-1-0-0,052CO,COPD
052CO-a_AAATGCCCACATCCGG-1-0-0-1-0-0,052CO,COPD
...,...,...
484C_TTGACTTTCATCTGCC-1-0-0-1,484C,control
484C_TTTACTGCAAATTGCC-1-0-0-1,484C,control
484C_TTTATGCCAATAGAGT-1-0-0-1,484C,control
484C_TTTATGCTCGATAGAA-1-0-0-1,484C,control


In [95]:
ad.obs

,GSM,sample.id,n_counts,log_counts,n_genes,mt_frac,patient.id,cell.type,study,disease.status,...,Diagnosis,COVID-19,Tissue Type,Cell Population,Cell Type,leiden,No of UMIs,No of genes,% of mito genes,Cluster
SC154IPFUP_AAACCTGCATCACGAT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,15616.0,9.656052,2600.0,0.093558,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGGTACCGGCT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,9768.0,9.186867,2008.0,0.084562,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAACGGGTCTGTCCGT-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,27659.0,10.227707,3509.0,0.043386,SC154,FABP4hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATCCCGC-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,9778.0,9.187890,2276.0,0.070157,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SC154IPFUP_AAAGATGAGATGCGAC-1-0-0-0-0-0-0,GSM3660658,SC154IPFUP,8610.0,9.060679,1981.0,0.059582,SC154,SPP1hi Macrophage,morse,IPF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484C_TTGACTTTCATCTGCC-1-0-0-1,NaN,NaN,16249.0,9.695786,3934.0,0.044187,484C,Macrophage_Outlier_D,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTACTGCAAATTGCC-1-0-0-1,NaN,NaN,1851.0,7.523481,1096.0,0.044300,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTATGCCAATAGAGT-1-0-0-1,NaN,NaN,11975.0,9.390576,3415.0,0.046180,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
484C_TTTATGCTCGATAGAA-1-0-0-1,NaN,NaN,6138.0,8.722254,2506.0,0.061584,484C,Macrophage,adams,control,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [96]:
ad.obs[ad.obs['study'] == 'adams'][['patient.id', 'disease.status']].drop_duplicates('patient.id')['disease.status'].value_counts()

IPF        32
control    28
COPD       18
Name: disease.status, dtype: int64

In [97]:
ad.obs[ad.obs['study'] == 'budinger'][['patient.id', 'disease.status']].drop_duplicates('patient.id')['disease.status'].value_counts()

covid    3
Name: disease.status, dtype: int64

In [98]:
ad.obs[ad.obs['study'] == 'BAL'][['patient.id', 'disease.status']].drop_duplicates('patient.id')['disease.status'].value_counts()

BAL (NA)    7
Name: disease.status, dtype: int64

In [99]:
ad.obs[ad.obs['study'] == 'morse'][['patient.id', 'disease.status']].drop_duplicates('patient.id')['disease.status'].value_counts()

control    8
IPF        8
Name: disease.status, dtype: int64

In [101]:
for k in ['MP.markers', 'MP.others']:
    print(k)
    next_marker_genes = marker_genes[k];
    sc.tl.score_genes(ad, list(next_marker_genes), use_raw=False, score_name=k + ".score");

MP.markers


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


MP.others


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/anndata/_core/anndata.py:1094: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if not is_categorical(df_full[k]):


In [70]:
ad[ad.obs['study'] == 'morse',:]

View of AnnData object with n_obs × n_vars = 17551 × 17656
    obs: 'GSM', 'sample.id', 'n_counts', 'log_counts', 'n_genes', 'mt_frac', 'patient.id', 'cell.type', 'study', 'disease.status', 'MP.markers.score', 'MP.others.score', 'MP.all.score', 'nUMI', 'nGene', 'CellType_Category', 'Manuscript_Identity', 'Subclass_Cell_Identity', 'Disease_Identity', 'Subject_Identity', 'Library_Identity', 'groups', 'batch', 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'viral_counts', 'viral_ratio', 'viral_norm', 'percent.mt', 'number', 'dataset', 'S.Score', 'G2M.Score', 'Phase', 'CC.Difference', 'SR.score1', 'dissoc.score1', 'Label', 'nCount_VIRAL', 'nFeature_VIRAL', 'seurat_clusters', 'RNA_mnn_40_Leiden_res.0.1', 'RNA_mnn_40_Leiden_res.0.2', 'RNA_mnn_40_Leiden_res.0.3', 'RNA_mnn_40_Leiden_res.0.4', 'RNA_mnn_40_Leiden_res.0.5', 'RNA_mnn_40_Leiden_res.0.6', 'RNA_mnn_40_Leiden_res.0.7', 'RNA_mnn_40_Leiden_res.0.8', 'RNA_mnn_40_Leiden_res.0.9', 'RNA_mnn_40_Leiden_res.1', 'RNA_mnn_40_Leiden_res.1.5', 'RNA_m

In [71]:
ad.obs['Subclass_Cell_Identity'].value_counts()

Macrophage                         74131
Macrophage_Alveolar                60551
nan                                47864
Macrophage_Outlier_B                8748
Macrophage_Outlier_C                4226
Macrophage_MT-tRNAs                 3759
Macrophage_Outlier_F                3498
Macrophage_Outlier_A                3485
Macrophage_Alveolar_CellCycle_A     3348
Macrophage_Alveolar_CellCycle_B     3013
Macrophage_CellCycle_B              2632
Macrophage_Alveolar_Outlier_B       2570
Macrophage_Outlier_HSPs             2415
Macrophage_Outlier_D                2190
Macrophage_Alveolar_Outlier_A       1382
Macrophage_CellCycle_A               886
Macrophage_Alveolar_Outlier_D        816
Macrophage_Alveolar_Outlier_C        752
Macrophage_Outlier_E                 472
Macrophage_Alveolar_Outlier_E        268
Macrophage_Alveolar_Outlier_F         58
Macrophage_CellCycle_MT-tRNAs         22
Name: Subclass_Cell_Identity, dtype: int64

In [72]:
adata.obs['study'].value_counts()

adams       179222
budinger     22810
morse        17551
BAL           7503
Name: study, dtype: int64

In [73]:
ad.obs.groupby(['study', 'disease.status']).size().unstack(fill_value=0)

disease.status,BAL (NA),COPD,IPF,control,covid
study,,,,,
BAL,7503,0,0,0,0
adams,0,30159,86002,63061,0
budinger,0,0,0,0,22810
morse,0,0,8301,9250,0


In [74]:
ad.obs['disease.status'] = ad.obs['disease.status'].str.replace('Control', 'control')
ad.obs['disease.status'] = np.where(ad.obs['disease.status'].str.contains('nan'), 'BAL', ad.obs['disease.status'])
ad.obs['cell.type'] = np.where(ad.obs['cell.type'].str.contains('nan'), ad.obs['Subclass_Cell_Identity'], ad.obs['cell.type'])
ad.obs['cell.type'] = np.where(ad.obs['cell.type'].str.contains('nan'), ad.obs['Celltype_2'], ad.obs['cell.type'])

In [75]:
ad.shape

(227086, 17656)

In [78]:
print('plotting...')
rcParams['figure.dpi'] = 110

fig, ax = plt.subplots(nrows = 2, ncols = 3, figsize=(18, 10))
ax = ax.flatten()

sc.pl.umap(ad, color='study', title='study', ax=ax[0], show=False)
ax[0].spines['right'].set_visible(False)
ax[0].spines['top'].set_visible(False)

sc.pl.umap(ad, color='patient.id', title='patient (n=%i)' % len(set(ad.obs['patient.id'])),
           ax=ax[1], show=False)
ax[1].spines['right'].set_visible(False)
ax[1].spines['top'].set_visible(False)
ax[1].get_legend().remove()
plt.xlabel('')
plt.ylabel('')

sc.pl.umap(ad, color='cell.type', title='cell type', ax=ax[2], show=False)
ax[2].spines['right'].set_visible(False)
ax[2].spines['top'].set_visible(False)
plt.xlabel('')
plt.ylabel('')

sc.pl.umap(ad, color='MP.markers.score' if 'study' in adata.obs else batch, title='score - 20 marker genes', ax=ax[3], show=False)
ax[3].spines['right'].set_visible(False)
ax[3].spines['top'].set_visible(False)
plt.xlabel('')
plt.ylabel('')

sc.pl.umap(ad, color='MP.others.score', title='score - 9 marker genes', ax=ax[4], show=False)
ax[4].spines['right'].set_visible(False)
ax[4].spines['top'].set_visible(False)
plt.xlabel('')
plt.ylabel('')

sc.pl.umap(ad, color='disease.status', title='disease.status', ax=ax[5], show=False)
ax[5].spines['right'].set_visible(False)
ax[5].spines['top'].set_visible(False)
plt.xlabel('')
plt.ylabel('')

plt.savefig('data/figures/umap_scvi_integrated_revision.pdf')
plt.close()

plotting...


In [81]:
os.path.abspath('data/figures/umap_scvi_integrated_revision.pdf')

'/storage/groups/ml01/workspace/ignacio.ibarra/theislab/saliba_macrophages/notebooks/data/figures/umap_scvi_integrated_revision.pdf'

In [82]:
print('done...')

done...


In [83]:
print('here...')

here...


In [86]:
from itertools import combinations

ad_sel = subset_anndata(ad, ad.shape[0])
disease = ad_sel.obs['disease.status']
conn = ad_sel.uns['neighbors']['connectivities']


masks = []
obs = []
masks = []
positions = np.arange(ad_sel.shape[0]) # .reshape(w, h)
# conn_array = conn.toarray() if (type(conn) != np.ndarray) else conn
conn_array = conn # conn.toarray() if (type(conn) != np.ndarray) else conn

pos_by_pair = {}

for i, j in combinations(list(set(disease))[::-1], r=2):
    x = positions[disease == i]
    y = positions[disease == j]

    
    xy = cartesian([x, y])
    x, y = xy[:, 0].flatten(), xy[:, 1].flatten()

    edges = conn_array[x, y]
    x_nodes = x[edges > 0]
    y_nodes = y[edges > 0]
    n_nodes_x, n_nodes_y = x_nodes.shape[0], y_nodes.shape[0]
    nx_uniq, ny_uniq = np.unique(x_nodes).shape[0], np.unique(y_nodes).shape[0]
    
    k = i + ':' + j
    if not k in pos_by_pair:
        pos_by_pair[i + ':' + j] = {}
    pos_by_pair[i + ':' + j][i] = np.unique(x_nodes)
    pos_by_pair[i + ':' + j][j] = np.unique(y_nodes)

    
    obs.append([i, j, nx_uniq, ny_uniq, nx_uniq + ny_uniq])

(227086, 17656)
adams       179222
budinger     22810
morse        17551
BAL           7503
Name: study, dtype: int64


/home/icb/ignacio.ibarra/miniconda3/envs/mypython3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: This location for 'connectivities' is deprecated. It has been moved to .obsp[connectivities], and will not be accesible here in a future version of anndata.
  """


IndexError: too many indices for array: array is 1-dimensional, but 2 were indexed

In [ ]:
df = pd.DataFrame(obs, columns=['a', 'b', 'n.a', 'n.b', 'n.total'])
df.sort_values('n.total', ascending=False)

In [ ]:
bal_ipf = ad_sel[pos_by_pair['BAL:IPF']['BAL'],:]
bal_ipf.obs['selection'] = 'BAL:IPF'
bal_ctrl = ad_sel[pos_by_pair['BAL:control']['BAL'],:]
bal_ctrl.obs['selection'] = 'BAL:CTRL'
print(bal_sel.obs['study'].value_counts())
print(bal_ctrl.obs['study'].value_counts())

bal_all = bal_ipf.concatenate(bal_ctrl)

### Plot whether the expression of gene markers for IPF+BAL increase after subsetting for IPFs

In [ ]:
bal_all.obs